In [ ]:
import pandas as pd
#
# https://medium.com/@duyanhnguyen_38925/create-a-strong-text-classification-with-the-help-from-elmo-e90809ba29da
# https://www.kaggle.com/saikumar587/text-classification-elmo

#import the train and test data
url = 'https://raw.githubusercontent.com/XinyanHe/413Dataset/master/stack.csv'
stack_data = pd.read_csv(url)
stack_data = stack_data.head(5000)

In [ ]:
#install ELMO
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow.compat.v1 as tf1
import numpy as np

tf1.disable_eager_execution()
tf1.disable_v2_behavior()

#download the elmo model
elmo = hub.Module("https://tfhub.dev/google/elmo/3", trainable=True)

def elmo_embedding(x):
    return elmo(tf.squeeze(tf.cast(x,tf.string)),signature='default',as_dict=True)["default"]

Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
from keras.layers import Input, Lambda, Dense
from keras.models import Model
import keras.regularizers

def build_model(): 
    input_text = Input(shape=(1,), dtype=tf.string)
    embedding = Lambda(elmo_embedding, output_shape=(1024, ))(input_text)
    dense = Dense(256, activation='relu')(embedding)
    pred = Dense(3, activation='softmax')(dense)
    model = Model(inputs=[input_text], outputs=pred)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
#decode and encode function

def encode(le_enc, labels):
    enc = le_enc.transform(labels)
    return keras.utils.to_categorical(enc)

def decode(le_enc, one_hot):
    dec = np.argmax(one_hot, axis=1)
    return le_enc.inverse_transform(dec)

In [ ]:
model_elmo = build_model()
model_elmo.summary()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
lambda (Lambda)              (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               262400    
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 771       
Total params: 263,171
Trainable params: 263,171
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from sklearn import metrics,preprocessing,model_selection
from sklearn.metrics import accuracy_score
from keras.utils import to_categorical

X = stack_data['Body'].tolist()
y = stack_data['Y'].tolist()

#fitting the model

le = preprocessing.LabelEncoder()
le.fit(y)

y_en = encode(le,y)
x_train, x_test, y_train, y_test = model_selection.train_test_split(np.asarray(X), np.asarray(y_en), test_size=0.2, random_state=42)

In [ ]:
with tf1.Session() as session:
    tf1.compat.v1.keras.backend.set_session(session)
    session.run(tf1.global_variables_initializer())  
    session.run(tf1.tables_initializer())
    history = model_elmo.fit(x_train, y_train, epochs=2, batch_size=16)
    model_elmo.save_weights('./response-elmo-model.h5')

Train on 4000 samples
Epoch 1/2
4000/4000 [==============================] - 7084s 2s/sample - loss: 0.7380 - acc: 0.6525
Epoch 2/2
4000/4000 [==============================] - 7405s 2s/sample - loss: 0.5297 - acc: 0.7638


Run the test set.

In [16]:
with tf1.Session() as session:
    tf1.compat.v1.keras.backend.set_session(session)
    session.run(tf1.global_variables_initializer())  
    session.run(tf1.tables_initializer())
    model_elmo.load_weights('./response-elmo-model.h5')  
    predicts  = model_elmo.predict(x_test, batch_size=16)

In [17]:
predicts = np.argmax(predicts, axis=1)
rounded_test=np.argmax(y_test, axis=1)

In [18]:
# decode predicted labels
print(metrics.classification_report(rounded_test, predicts))

              precision    recall  f1-score   support

           0       0.74      0.88      0.80       416
           1       0.78      0.56      0.65       335
           2       0.88      0.94      0.91       249

    accuracy                           0.79      1000
   macro avg       0.80      0.79      0.79      1000
weighted avg       0.79      0.79      0.78      1000

